In [1]:
!pip install mysql-connector-python

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df
pd.set_option('display.max_rows', None)     # es para enseñar todas las filas del df


import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

In [3]:
import mysql.connector as conn

# Importo todos archivos, exploro y modifico

### ACTOR

In [4]:
df_actor=pd.read_csv('../data/actor.csv')


In [5]:
# Elimino la columna 'last_update'

df_actor = df_actor.drop(['last_update'], axis=1)

In [6]:
df_actor.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   actor_id    200 non-null    int64 
 1   first_name  200 non-null    object
 2   last_name   200 non-null    object
dtypes: int64(1), object(2)
memory usage: 26.2 KB


In [7]:
len(df_actor.drop_duplicates())==len(df_actor)  # no hay duplicados

True

### CATEGORY

In [8]:
df_category=pd.read_csv('../data/category.csv')

In [9]:
# Elimino la columna 'last_update'

df_category = df_category.drop(['last_update'], axis=1)

In [10]:
# Crear una fila nueva con una nueva categoría.

df2 = pd.DataFrame({'category_id': [99],
                    'name' : ['Unknown']})

df_category = pd.concat([df_category, df2], ignore_index = True, axis = 0)

### FILM

In [11]:
df_film=pd.read_csv('../data/film.csv')


### INVENTORY

In [12]:
df_inventory=pd.read_csv('../data/inventory.csv')

In [13]:
# Elimino la columna 'last_update'

df_inventory = df_inventory.drop(['last_update'], axis=1)

### LANGUAGE

In [14]:
df_language=pd.read_csv('../data/language.csv')


In [15]:
# Elimino la columna 'last_update'

df_language = df_language.drop(['last_update'], axis=1)

### RENTAL

In [16]:
df_rental=pd.read_csv('../data/rental.csv')

In [17]:
# Cambio el 'return_date' a formato 'date'

df_rental.rental_date = pd.to_datetime(df_rental.rental_date, format= '%Y-%m-%d')
df_rental.return_date = pd.to_datetime(df_rental.return_date, format= '%Y-%m-%d')

In [18]:
# Elimino la columna 'last_update'

df_rental = df_rental.drop(['last_update'], axis=1)

In [19]:
# Inserto numeros random en  la columna 'inventory_id'


data_random = np.random.randint(1, 1000, size=1000)
df_rental.inventory_id = data_random

### OLD_HDD - Relaciones titulo, actor y categoría

In [20]:
df_old_HDD=pd.read_csv('../data/old_HDD.csv')

In [21]:
df_old_HDD.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 207.6 KB


# Modificar y limpiar

## Modifico tabla film

In [22]:
# Hago un left join en 'df_film' para obtener 'category_id', haciendo que en films y old_HDD el index sea title.


film = df_film.set_index('title').join(df_old_HDD[["title", "category_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


In [23]:
# Le elimino duplicado que se generan

film.drop_duplicates(inplace=True)

In [24]:
# Le reseteo el indice

film.reset_index(inplace=True)

In [25]:
# Relleno en la tabla film la columna category_id con 99 

film.category_id.fillna(99, inplace=True)

# Modificar la columna de category_id

film.category_id = film.category_id.astype(dtype='int64')

In [26]:
# Elimino la columna de original_language_id

film.drop('original_language_id', axis=1, inplace=True)

In [27]:
# Elimino la columna 'last_update'

film = film.drop(['last_update'], axis=1)

## Creo tabla actor_film

In [28]:
actor = df_actor.copy(deep=True)

In [29]:
# Creo una columna y concateno first name y last name

actor['name'] = actor['first_name'] + ' ' + actor['last_name']

In [30]:
hdd = df_old_HDD

In [31]:
# Creo una columna nueva y concateno first name y last name

hdd['name'] = hdd['first_name'] + ' ' + hdd['last_name']

In [32]:
# Creo la tabla 'actor_film' que voy a utilizar como relación many to many entre actor y film.

# Primero uso hdd para hacer un join a través del indice 'title' y obtener 'film_id'

actor_film = hdd.set_index('title').join(film[["title", "film_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


In [33]:
# Elimino duplicados por si se generaran

actor_film.drop_duplicates(inplace=True)

In [34]:
# Reseteo el indice para que la columna 'title' vuelva a ser una columna

actor_film.reset_index(inplace=True)

In [35]:
# Primero uso 'actor_film' para hacer un join a través del indice 'name' y obtener 'actor_id'

actor_film = actor_film.set_index('name').join(actor[["name", "actor_id"]].set_index('name'), lsuffix='', rsuffix='_dcha', how='left')

In [36]:
# Elimino duplicados y reseteo el índice

actor_film.drop_duplicates(inplace=True)
actor_film.reset_index(inplace=True)

In [37]:
# Sobreescribo 'actor_film' solo con las columnas de 'actor_id' y 'film_id'

actor_film = actor_film[['actor_id', 'film_id']]

## Creo e importo tablas nuevas

In [38]:
# Creo 'df_store' dataframe

store = {'store_id': [1, 2], 'name': ['Blockbuster MIA','Blockbuster Orlando' ], 'city': ['Miami', 'Orlando']}  
  
df_store = pd.DataFrame(store)

In [39]:
# Creo 'df_staff' dataframe

staff = {'staff_id': [1, 2], 'firstname': ['Barack','Donald' ], 'lastname': ['Obama', 'Trump'], 'salary': [35000, 35000]}  
  
df_staff = pd.DataFrame(staff)  


In [40]:
# Importo un random csv de 'customers' y me creo tabla 'id_customer' 

df_customer=pd.read_csv('../data/customers.csv', sep=';')

df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  1000 non-null   int64 
 1   firstname    1000 non-null   object
 2   lastname     1000 non-null   object
 3   email        1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


## Exporto todas los dataframe a carpeta new_data

In [41]:
df_category.to_csv('../data/new_data/category.csv', sep= ",", index=False)

df_language.to_csv('../data/new_data/language.csv', sep= ",", index=False)

df_actor.to_csv('../data/new_data/actor.csv', sep= ",", index=False)

actor_film.to_csv('../data/new_data/actor_film.csv', sep= ",", index=False)

film.to_csv('../data/new_data/film.csv', sep= ",", index=False)

df_inventory.to_csv('../data/new_data/inventory.csv', sep= ",", index=False)

df_rental.to_csv('../data/new_data/rental.csv', sep= ",", index=False)

df_store.to_csv('../data/new_data/store.csv', sep= ",", index=False)

df_customer.to_csv('../data/new_data/customer.csv', sep= ",", index=False)


df_staff.to_csv('../data/new_data/staff.csv', sep= ",", index=False)